In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f'''
CREATE TABLE IF NOT EXISTS {catalog_name}.dp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_current_his
(
  calday DATE,
  promotion_id STRING,
  bonus_buy_id STRING,
  bonus_buy_note STRING,
  bonus_buy_profile STRING,
  promo_start_date DATE,
  promo_end_date DATE,
  store_id STRING,
  supply_region STRING,
  buy_item_id STRING,
  discount_product_id STRING,
  value_claimback NUMERIC,
  form_claimback STRING,
  vendor_free_goods STRING,
  vendor_price STRING,
  block_sto BIGINT,
  block_po BIGINT
)
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
''')

In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_current_his
WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)-1;    
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_current_his
SELECT 
DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) -1 calday,
    promotion_id,
    bonus_buy_id,
    bonus_buy_note,
    bonus_buy_profile,
    promo_start_date,
    promo_end_date,
    store_id,
    CAST(NULL AS STRING) supply_region,
    buy_item_id,
    discount_product_id,
    value_claimback,
    form_claimback,
    CAST(NULL AS STRING) vendor_free_goods,
    CAST(NULL AS STRING) vendor_price,
    block_sto,
    block_po
FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_winare_auto_block_promotion_current
""")